In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import classification_report
from sklearn.metrics import pairwise_distances_argmin_min

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="e5af6275806e4e8ebf14a5d71e202fad",
                                                           client_secret="a72673ed83bb437d83c801b760a066d5"))

In [18]:
def get_df_from_playlist(playlist_id):
    
    results = sp.user_playlist_tracks("spotify", playlist_id)
    tracks = results['items']

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    track_ids = [track['track']['uri'] for track in tracks]
    
    start = 0
    df = pd.DataFrame()

    for stop in range(0, len(track_ids)+100, 100):
        if start != stop:
            new_df = pd.json_normalize(sp.audio_features(track_ids[start:stop]))
            df = pd.concat([new_df, df])
            start = stop

    df = df.reset_index(drop=True)
    df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url',], axis=1, inplace=True)
    
    main_artist_names = [track['track']['artists'][0]['name'] for track in tracks]
    song_names = [track['track']['name'] for track in tracks]
    df['main_artist'] = main_artist_names
    df['song_name'] = song_names

    return df

In [19]:
#official Billboard Hot 100 playlist

df_hot100_current = get_df_from_playlist("6UeSakyzhiEt4NB3UAd6NQ")

In [ ]:
#From Wikipedia: List of Billboard Hot 100 top-ten singles in 2023

#r = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_2023')
#r.status_code

In [30]:
#official Spotify top 50 charts: Global and top 10 markets
#https://en.wikipedia.org/wiki/List_of_largest_recorded_music_markets (2022 table)

top_by_market_playlists = {
    
    "top_gb" : "37i9dQZEVXbNG2KDcFcKOF", #Global
    "top_us" : "37i9dQZEVXbLp5XoPON0wI", #United States
    "top_jp" : "37i9dQZEVXbKqiTGXuCOsB", #Japan
    "top_uk" : "37i9dQZEVXbMwmF30ppw50", #United Kingdom
    "top_de" : "37i9dQZEVXbK8BKKMArIyl", #Germany
    "top_fr" : "37i9dQZEVXbIPWwFssbupI", #France
    "top_sk" : "37i9dQZEVXbJZGli0rRP3r", #South Korea
    "top_ca" : "37i9dQZEVXbMda2apknTqH", #Canada
    "top_br" : "37i9dQZEVXbKzoK95AbRy9", #Brazil
    "top_au" : "37i9dQZEVXbK4fwx2r07XW", #Australia
    "top_it" : "37i9dQZEVXbJUPkgaWZcWG" #Italy (no playlist available for China)
}

In [33]:
df_top_all_markets = pd.DataFrame()

for pl in top_by_market_playlists:
    
    df_ = get_df_from_playlist(top_by_market_playlists[pl])
    df_top_all_markets = pd.concat([df_top_all_markets,df_]).drop_duplicates().reset_index(drop=True)

df_top_all_markets

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,main_artist,song_name
0,0.868,0.538,5.0,-8.603,1.0,0.1740,0.26900,0.000003,0.0901,0.732,99.968,231750.0,4.0,Doja Cat,Paint The Town Red
1,0.802,0.832,11.0,-4.107,1.0,0.0434,0.31100,0.000000,0.0815,0.890,124.997,185427.0,4.0,Jung Kook,Seven (feat. Latto) (Explicit Ver.)
2,0.511,0.532,5.0,-5.745,1.0,0.0578,0.17700,0.000000,0.2910,0.350,138.005,219724.0,4.0,Olivia Rodrigo,vampire
3,0.552,0.702,9.0,-5.707,1.0,0.1570,0.11700,0.000021,0.1050,0.564,169.994,178427.0,4.0,Taylor Swift,Cruel Summer
4,0.627,0.879,9.0,-3.446,1.0,0.0955,0.00193,0.000007,0.0647,0.748,129.979,184784.0,4.0,Olivia Rodrigo,bad idea right?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,0.740,0.856,6.0,-5.385,0.0,0.1270,0.12200,0.000115,0.1840,0.467,134.957,139099.0,4.0,Vale Pain,In Mezzo ai Guai (feat. Neima Ezza)
366,0.716,0.737,6.0,-5.831,0.0,0.0578,0.11500,0.000000,0.0922,0.584,131.947,179864.0,4.0,Geolier,CHIAGNE (feat. Lazza & Takagi & Ketra)
367,0.802,0.712,9.0,-7.204,0.0,0.4230,0.13900,0.000000,0.0750,0.515,101.896,133859.0,4.0,Rhove,Pelé
368,0.458,0.648,6.0,-8.372,1.0,0.0974,0.22800,0.000000,0.0892,0.234,80.835,181853.0,3.0,Tedua,Malamente


In [ ]:
#most popular music genres
#https://headphonesaddict.com/music-genre-statistics/#Most_Popular_Music_Genre_Statistics

#top_genres_gb = ['Pop', 'Rock', '90s', '80s', 'Soundtracks', 'Hip-hop/Rap/Trap', 'R&B', 'Dance/Electronic/House', '70s', 'Soul/Blues']

In [ ]:
#most popular music genres
#spotify playlists

#"most streamed rap songs on spotify"